In [ ]:
import preamble

# Constants

In [ ]:
import numpy as np

# Experiment
USER_ID = "11_P=0,V=0"

# Trajectory Planner
REFERENCE_TRAJECTORY_CSV_PATH = '../data/reference_path.csv'
INIT_POSE = np.array([-0.54, -0.27, 0.0 + 0.02, -1.21, -1.21, 1.25]) # Also used for Robot

# Transition Probabilities
state_map = {(0, 0): 0, (1, 0): 1, (0, 1): 2, (1, 1): 3}
A = np.array([
    [ 0.7180186,  0.6531676,  0.9194265, -1.1720577],
    [ 0.5751301,  0.7693612, -1.3269429,  0.6973043],
    [ 0.8416619, -1.0936067,  0.7530234,  0.6233168],
    [-2.9374210,  0.3163404,  0.3909466,  0.2452125]
])
B = np.array([
    [ 0.4821496, -0.3078547,  0.4216300, -1.8944792],
    [ 0.6335048,  0.3842422, -1.8434653,  0.4650253],
    [ 1.0996757, -1.8577393,  0.5245419,  0.3539045],
    [-2.3862197,  0.7957043, -0.5840249, -0.5731659],
])
C = np.array([
    [ 3.333601, -1.9594908, -3.6424782, -3.0082774],
    [-2.864783,  2.3264263, -3.7144790, -2.5864382],
    [-3.484102, -3.6938949,  3.1255682, -1.5355915],
    [-4.482538, -4.6248226, -2.1531882,  4.6947365],
])
prior_belief = np.full(4, 0.25)
U_p, U_v = 0, 0
P_hat, V_hat = 0.5, 0.5
dVdt = 0
DVDT_ENCOURAGEMENT_THRESHOLD = 0.01
BAYESIAN_FILTER_SAMPLING_RATE = 100

# Prior Distributions
# MEANS = np.array([
#     [-3.99849367, -3.16462103],
#     [-3.87261021, -3.14348198],
#     [-4.14477961, -3.16904538],
#     [-4.11880238, -3.24909682],
# ])
# COVARIANCES = np.array([
#     [[0.30300593, 0.04605989], [0.04605989, 0.55695469]],
#     [[0.30524614, 0.08586392], [0.08586392, 0.58394479]],
#     [[0.30183493, 0.0819836 ], [0.0819836 , 0.56152217]],
#     [[0.3647403 , 0.11946842], [0.11946842, 0.5560613 ]],
# ])

# Prior Distributions
# MEANS = np.array([
#     [-2.78253669, -2.82148201],  # 00
#     [-3.67830111, -3.04871910],  # 10
#     [-3.15370674, -2.96044470],  # 01
#     [-4.02882005, -3.94904296],  # 11
# ])
# COVARIANCES = np.array([
#     [[0.99763112, 0.19855546],
#      [0.19855546, 0.51791682]],  # 00
#     [[1.98675137, 0.43011312],
#      [0.43011312, 0.70954010]],  # 10
#     [[0.86434026, 0.18075753],
#      [0.18075753, 0.55608555]],  # 01
#     [[0.95104567, 0.18935949],
#      [0.18935949, 0.63709599]],  # 11
# ])

# Prior Distributions
MEANS = np.array([
    [-2.78253669, -2.82148201,  2.31171387],  # 00
    [-3.47830111, -3.04871910,  1.46494315],  # 10
    [-3.45370674, -2.96044470,  1.50144656],  # 01
    [-4.02882005, -3.94904296,  1.07797773],  # 11
])
COVARIANCES = np.array([
    [[0.99763112, 0.19855546, 0.70999041],
     [0.19855546, 0.51791682, 0.26067248],
     [0.70999041, 0.26067248, 0.96295218]],  # 00

    [[1.98675137, 0.43011312, 1.32838066],
     [0.43011312, 0.70954010, 0.64255399],
     [1.32838066, 0.64255399, 1.68856655]],  # 10

    [[0.86434026, 0.18075753, 0.47364069],
     [0.18075753, 0.55608555, 0.32541218],
     [0.47364069, 0.32541218, 0.70555740]],  # 01

    [[0.95104567, 0.18935949, 0.44941941],
     [0.18935949, 0.63709599, 0.18373808],
     [0.44941941, 0.18373808, 0.46763161]],  # 11
])

# Verbal PD
VERBAL_PD_SCALER = 2
K_v = 5.6 * VERBAL_PD_SCALER
B_v = 4.8 * VERBAL_PD_SCALER

# Physical PD
PHYSICAL_PD_SCALER = 3
K_p = 19 * PHYSICAL_PD_SCALER
B_p = 11 * PHYSICAL_PD_SCALER

# F2L
F2L_MODEL_PATH = '../data/F2L_models/svm_knn.pkl'
T_proj = 4
N_proj = 256
dt_proj = T_proj / N_proj
FINAL_IMPULSE_THRESHOLD = 1.0

# System dynamics
M = 2 * 2.0
B = 17.7

# Real time data collection
TABLE_COLUMN_NAMES = [
    't',
    'x',
    'x_dot',
    'F_h',
    'ref_going_forward',
    'x_ref',
    'x_dot_ref',
    'e',
    'e_dot',
    'P_hat',
    'V_hat',
    'c_p',
    'c_v',
    'verbal_P_tracking_term',
    'verbal_D_tracking_term',
    'verbal_PD_tracking_term',
    'A_v',
    'U_v',
    'physical_P_tracking_term',
    'physical_D_tracking_term',
    'physical_PD_tracking_term',
    'A_p',
    'U_p',
    'successful_utterance',
    'is_speaking',
    'phrase',
    'U',
    'human_disturbance',
    'system_dynamics_input',
]
DATA_COLLECTION_SAMPLING_RATE = 200

# Real time data plotting
REAL_TIME_DATA_PLOTTING_ENABLED = False
PLOTTING_SAMPLING_RATE = 30
DISPLAY_TIME_WINDOW = 5

USE_CAMERA = True

# Robot
ROBOT_IP = '169.254.9.43'
FORCE_EWMA_TAU = 0.75
TRANSLATIONAL_FORCE_DEADBAND = 2

# Experiment

This baseline is made up of all componentes: Physical PD, Physical Admittance, Verbal PD, Verbal Admittance, F2L, Compliance Estimator, and $J$ Optimization Method.

In [ ]:
from timer import Timer
from trajectory_planner import TrajectoryPlanner
from model import SVMKNNModel
import pickle
from vocalizer import Vocalizer
from virtual_dynamics import SimpleVirtualDynamics
from tabular_data_store import TabularDataStore
from plot_client import PlotClient
from robot import Robot
from console_command_thread import ConsoleCommandThread
import scipy.stats
from phrase_mapping import MEAN_WORDS_PER_SIMPLE_PHRASE, SIMPLE_PHRASE_MAPPING
from rgbd_stream import RGBDStream_iOS
from camera_feed import CameraFeed
from scipy.linalg import expm, logm
import random
import time

#############
### SETUP ###
#############

### Experiment ###
experiment_timer = Timer()

### Trajectory Planner ###
traj = TrajectoryPlanner(REFERENCE_TRAJECTORY_CSV_PATH, INIT_POSE[:3])

### Compliance Estimator ###
compliance_estimator_timer = Timer()

### F2L ###
with open(F2L_MODEL_PATH, 'rb') as file:
    F2L_model: SVMKNNModel = pickle.load(file)
phrase = ''
voc = Vocalizer()
speaking_timer = Timer()
speaking_start_t = 0.0
speaking_start_V_hat = 1.0

### System dynamics ###
vd = SimpleVirtualDynamics(M=M, B=B, K=0)

### Real time data collection ###
table = TabularDataStore(column_names=TABLE_COLUMN_NAMES)
table_timer = Timer()

### Real time data plotting ###
if REAL_TIME_DATA_PLOTTING_ENABLED:
    plot = PlotClient()
    
    plot.create_plot('01_e', title='$e$ vs Time', xlabel='Time (s)', ylabel='Displacement (m)')
    plot.create_line("01_e", "e_x", color="r", label="$e_x$")
    plot.create_line("01_e", "e_y", color="g", label="$e_y$")
    plot.create_line("01_e", "e_z", color="b", label="$e_z$")

    plot.create_plot('02_e_norm', title='$||e||$ vs Time', xlabel='Time (s)', ylabel='Displacement (m)')
    plot.create_line("02_e_norm", "e_norm", label="$||e||$")

    plot.create_plot('03_e_log_norm', title='$\log||e||$ vs Time', xlabel='Time (s)', ylabel='Log Displacement ($\log$ m)')
    plot.create_line("03_e_log_norm", "e_log_norm", label="$\log||e||$")

    plot.create_plot('04_e_dot', title='$\dot{e}$ vs Time', xlabel='Time (s)', ylabel='Velocity (m/s)')
    plot.create_line("04_e_dot", "e_dot_x", color="r", label="$\dot{e}_x$")
    plot.create_line("04_e_dot", "e_dot_y", color="g", label="$\dot{e}_y$")
    plot.create_line("04_e_dot", "e_dot_z", color="b", label="$\dot{e}_z$")

    plot.create_plot('05_e_dot_norm', title='$||\dot{e}||$ vs Time', xlabel='Time (s)', ylabel='Velocity (m/s)')
    plot.create_line("05_e_dot_norm", "e_dot_norm", label="$||\dot{e}||$")

    plot.create_plot('06_e_dot_log_norm', title='$\log||\dot{e}||$ vs Time', xlabel='Time (s)', ylabel='Log Velocity ($\log$ m/s)')
    plot.create_line("06_e_dot_log_norm", "e_dot_log_norm", label="$\log||\dot{e}||$")

    plot.create_plot('07_compliance', title='Estimated Compliance vs Time', xlabel='Time (s)', ylabel='Compliance [0, 1]', ylim=[0,1])
    plot.create_line("07_compliance", "P_hat", color="orange", label="$\hat{P}$")
    plot.create_line("07_compliance", "V_hat", color="purple", label="$\hat{V}$")

    plot.create_plot("08_J_v_proj", title="Projected $J_v$ Profile", xlabel="Future Time (s)", ylabel="Impulse (Ns)")
    plot.create_line("08_J_v_proj", "J_v_x", color="r", label="$J_{v,x}$")
    plot.create_line("08_J_v_proj", "J_v_y", color="g", label="$J_{v,y}$")
    plot.create_line("08_J_v_proj", "J_v_z", color="b", label="$J_{v,z}$")

    plot.create_plot('09_U_v', title='$U_v$ vs Time', xlabel='Time (s)', ylabel='$U_v$ (N)')
    plot.create_line("09_U_v", "U_v_x", color="r", label="$U_{v,x}$")
    plot.create_line("09_U_v", "U_v_y", color="g", label="$U_{v,y}$")
    plot.create_line("09_U_v", "U_v_z", color="b", label="$U_{v,z}$")

    plot.create_plot('10_U_p', title='$U_p$ vs Time', xlabel='Time (s)', ylabel='$U_p$ (N)')
    plot.create_line("10_U_p", "U_p_x", color="r", label="$U_{p,x}$")
    plot.create_line("10_U_p", "U_p_y", color="g", label="$U_{p,y}$")
    plot.create_line("10_U_p", "U_p_z", color="b", label="$U_{p,z}$")

    plot.create_plot('11_U_norm', title='$||U||$ vs Time', xlabel='Time (s)', ylabel='$||U||$ (N)')
    plot.create_line("11_U_norm", "U_p_norm", color="orange", label="$||U_p||$")
    plot.create_line("11_U_norm", "U_v_norm", color="purple", label="$||U_v||$")

    plot_timer = Timer()

if USE_CAMERA:
    rgbd_stream = RGBDStream_iOS()
    calibration_matrix = np.load('calibration_matrix.npy')
    camera_feed = CameraFeed('Camera Feed', rgbd_stream, calibration_matrix)

    yellow_dot_timer = Timer()
    while yellow_dot_timer.t() < 7.0:
        camera_feed.draw_world_point(INIT_POSE[:3], radius=10, color=(0xff, 0xff, 0))
        camera_feed.update_window()

z = None

### Robot ###
with Robot(
    ROBOT_IP,
    default_axes=Robot.TRANSLATION,
    init_pose=INIT_POSE,
    force_ewma_tau=FORCE_EWMA_TAU,
    translational_force_deadband=TRANSLATIONAL_FORCE_DEADBAND,
) as r, ConsoleCommandThread() as c:

    ##################
    ### EXPERIMENT ###
    ##################

    experiment_timer.reset()
    compliance_estimator_timer.reset()
    speaking_timer.reset()

    while c.is_alive():
        period_start = r.control.initPeriod()

        ### Current state ###
        t = experiment_timer.t()
        dt = experiment_timer.dt()
        x = r.get_pose()
        x_dot = r.get_velocity()
        F_h = r.get_force()

        ### Trajectory Planner ###
        ref_going_forward = traj.update_reference_trajectory(x)
        x_ref, x_dot_ref = traj.get_closest_target(x)

        ### Tracking errors ###
        e = x_ref - x
        e_dot = x_dot_ref - x_dot

        ### Compliance Estimator ###
        if compliance_estimator_timer.t() >= BAYESIAN_FILTER_SAMPLING_RATE**(-1):
            z = np.log(np.array([np.linalg.norm(e), np.linalg.norm(e_dot), np.linalg.norm(F_h - U_v)]) + 1e-10)

            # Transition Update
            U_p_norm = 0.2 * np.linalg.norm(U_p)
            U_v_norm = int(voc.is_uttering())

            logits = A * U_p_norm + B * U_v_norm + C

            logits_stable = logits - np.max(logits, axis=1, keepdims=True)
            exp_logits = np.exp(logits_stable)
            T = exp_logits / exp_logits.sum(axis=1, keepdims=True)
            T = expm(dt / 0.01 * logm(T))

            predict_belief = T.T @ prior_belief

            # Observation Update
            likelihoods = np.zeros(4)
            for i in range(4):
                likelihoods[i] = scipy.stats.multivariate_normal.pdf(z, mean=MEANS[i], cov=COVARIANCES[i] * 0.005 / dt)
            posterior_belief = predict_belief * likelihoods
            posterior_belief = np.full(4, 0.25) if posterior_belief.sum() == 0.0 else posterior_belief / posterior_belief.sum()
            prior_belief = posterior_belief

            # Marginalize Joint State
            previous_V_hat = V_hat
            alpha = 1.0 - np.exp(-2.0 * dt)
            P_hat = P_hat * (1.0 - alpha) + (posterior_belief[state_map[(1, 0)]] + posterior_belief[state_map[(1, 1)]]) * alpha
            V_hat = V_hat * (1.0 - alpha) + (posterior_belief[state_map[(0, 1)]] + posterior_belief[state_map[(1, 1)]]) * alpha
            dVdt = (V_hat - previous_V_hat) / dt

            compliance_estimator_timer.reset()

        ### Guidance costs ###
        c_p = (P_hat + V_hat) / 2
        c_v = 1 - V_hat

        ### Verbal PD ###
        verbal_P_tracking_term = K_v * e
        verbal_D_tracking_term = B_v * e_dot
        verbal_PD_tracking_term = verbal_P_tracking_term + verbal_D_tracking_term

        ### Verbal Admittance ###
        A_v = c_p / (c_p + c_v)
        U_v = verbal_PD_tracking_term - A_v*F_h

        ### Physical PD ###
        physical_P_tracking_term = K_p * e
        physical_D_tracking_term = B_p * e_dot
        physical_PD_tracking_term = physical_P_tracking_term + physical_D_tracking_term

        ### Physical Admittance ###
        A_p = c_v / (c_p + c_v)
        U_p = physical_PD_tracking_term - A_p*F_h

        ### F2L ###
        x_sim = np.copy(x)
        x_dot_sim = np.copy(x_dot)

        U_v_profile = []

        for i in range(N_proj):
            x_ref_proj, x_dot_ref_proj = traj.get_closest_target(x_sim)
            
            e_proj = x_ref_proj - x_sim
            e_dot_proj = x_dot_ref_proj - x_dot_sim

            U_v_proj = K_v * e_proj + B_v * e_dot_proj
            U_v_profile.append(U_v_proj)

            U_p_proj = K_p * e_proj + B_p * e_dot_proj

            U_proj = U_v_proj

            a_proj = (U_proj - B * x_dot_sim) / M
            x_dot_sim += a_proj * dt_proj
            x_sim += x_dot_sim * dt_proj

        U_v_profile = np.array(U_v_profile)
        impulse_curve = U_v_profile.cumsum(axis=0) * dt_proj
        final_impuse = impulse_curve[-1, :]

        speaking_period = MEAN_WORDS_PER_SIMPLE_PHRASE / np.exp(0.57 - 0.80 * (P_hat + V_hat))

        if not voc.is_uttering():
            if t - speaking_start_t > 4.0 and t - speaking_start_t < 4.5 and (V_hat - speaking_start_V_hat > 0.05 or V_hat > 0.95):
                phrase = 'good job'
            elif speaking_timer.t() <= speaking_period or np.linalg.norm(final_impuse) < FINAL_IMPULSE_THRESHOLD:
                phrase = ''
            else:
                words = F2L_model.force_to_phrase(impulse_curve[None, :])[0]
                phrase = random.choice(SIMPLE_PHRASE_MAPPING[words[-1]])

                speaking_timer.reset()
                speaking_start_t = t
                speaking_start_V_hat = V_hat

        successful_utterance = voc.utter(phrase)

        is_speaking = voc.is_uttering()

        ### System dynamics ###
        U = U_p + U_v
        human_disturbance = F_h - U_v
        system_dynamics_input = U + human_disturbance
        vd.apply_force(system_dynamics_input, dt)
        r.set_velocity(vd.get_velocity(), acceleration=10)

        ### Real time data collection ###
        if table_timer.t() >= DATA_COLLECTION_SAMPLING_RATE**(-1):
            table.append_row((
                t,
                x,
                x_dot,
                F_h,
                ref_going_forward,
                x_ref,
                x_dot_ref,
                e,
                e_dot,
                P_hat,
                V_hat,
                c_p,
                c_v,
                verbal_P_tracking_term,
                verbal_D_tracking_term,
                verbal_PD_tracking_term,
                A_v,
                U_v,
                physical_P_tracking_term,
                physical_D_tracking_term,
                physical_PD_tracking_term,
                A_p,
                U_p,
                successful_utterance,
                is_speaking,
                phrase,
                U,
                human_disturbance,
                system_dynamics_input,
            ))
            
            table_timer.reset()

        ### Real time data plotting ###
        if REAL_TIME_DATA_PLOTTING_ENABLED:
            if plot_timer.t() >= PLOTTING_SAMPLING_RATE**(-1):
                plot.update_line("01_e", "e_x", (t, e[0]))
                plot.update_line("01_e", "e_y", (t, e[1]))
                plot.update_line("01_e", "e_z", (t, e[2]))
                plot.config_plot("01_e", xlim=(t - DISPLAY_TIME_WINDOW, t))

                plot.update_line("02_e_norm", "e_norm", (t, np.linalg.norm(e)))
                plot.config_plot("02_e_norm", xlim=(t - DISPLAY_TIME_WINDOW, t))

                plot.update_line("03_e_log_norm", "e_log_norm", (t, np.log(np.linalg.norm(e))))
                plot.config_plot("03_e_log_norm", xlim=(t - DISPLAY_TIME_WINDOW, t))

                plot.update_line("04_e_dot", "e_dot_x", (t, e_dot[0]))
                plot.update_line("04_e_dot", "e_dot_y", (t, e_dot[1]))
                plot.update_line("04_e_dot", "e_dot_z", (t, e_dot[2]))
                plot.config_plot("04_e_dot", xlim=(t - DISPLAY_TIME_WINDOW, t))

                plot.update_line("05_e_dot_norm", "e_dot_norm", (t, np.linalg.norm(e_dot)))
                plot.config_plot("05_e_dot_norm", xlim=(t - DISPLAY_TIME_WINDOW, t))

                plot.update_line("06_e_dot_log_norm", "e_dot_log_norm", (t, np.log(np.linalg.norm(e_dot))))
                plot.config_plot("06_e_dot_log_norm", xlim=(t - DISPLAY_TIME_WINDOW, t))

                plot.update_line("07_compliance", "P_hat", (t, P_hat))
                plot.update_line("07_compliance", "V_hat", (t, V_hat))
                plot.config_plot("07_compliance", xlim=(t - DISPLAY_TIME_WINDOW, t))

                timesteps = np.arange(len(U_v_profile)) * dt_proj
                plot.update_line("08_J_v_proj", "J_v_x", (timesteps, U_v_profile[:, 0]), mode="replace")
                plot.update_line("08_J_v_proj", "J_v_y", (timesteps, U_v_profile[:, 1]), mode="replace")
                plot.update_line("08_J_v_proj", "J_v_z", (timesteps, U_v_profile[:, 2]), mode="replace")
                # No sliding window for projection plot

                plot.update_line("09_U_v", "U_v_x", (t, U_v[0]))
                plot.update_line("09_U_v", "U_v_y", (t, U_v[1]))
                plot.update_line("09_U_v", "U_v_z", (t, U_v[2]))
                plot.config_plot("09_U_v", xlim=(t - DISPLAY_TIME_WINDOW, t))

                plot.update_line("10_U_p", "U_p_x", (t, U_p[0]))
                plot.update_line("10_U_p", "U_p_y", (t, U_p[1]))
                plot.update_line("10_U_p", "U_p_z", (t, U_p[2]))
                plot.config_plot("10_U_p", xlim=(t - DISPLAY_TIME_WINDOW, t))

                plot.update_line("11_U_norm", "U_p_norm", (t, np.linalg.norm(U_p)))
                plot.update_line("11_U_norm", "U_v_norm", (t, np.linalg.norm(U_v)))
                plot.config_plot("11_U_norm", xlim=(t - DISPLAY_TIME_WINDOW, t))

                plot_timer.reset()

        if USE_CAMERA:
            p_r = x + U_p / (np.linalg.norm(U_p) + 1e-6) * 2.0 * 0.075 * np.log(np.linalg.norm(U_p) + 1.0)
            p_h = x + F_h / (np.linalg.norm(F_h) + 1e-6) * 2.0 * 0.075 * np.log(np.linalg.norm(F_h) + 1.0)
            camera_feed.draw_world_arrow(x - np.array([0.0, 0.0, 0.025]), p_r - np.array([0.0, 0.0, 0.025]), thickness=6, color=(0x3C / 0xFF, 0x91 / 0xFF, 0xE6 / 0xFF))
            camera_feed.draw_world_arrow(x, p_h, thickness=6, color=(0xFF / 0xFF, 0x87 / 0xFF, 0x1F / 0xFF))
            camera_feed.update_window()

        r.control.waitPeriod(period_start)

table.to_pandas().to_pickle(f'../data/experiments/B2/{USER_ID}_{time.time()}.pkl')